In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import re
sys.path.insert(0,"..")
from src.evaluation.evaluate import evaluate_df,evaluate_sentence
from src.data.make_dataset import recompose, additional_cleaners
from camel_tools.utils.charsets import UNICODE_PUNCT_CHARSET
from src.predict.mle_predict import apply_mle, load_mle_model, tokenize_skiphyph
from src.predict import translit_rules
from src.train.mle_train import train_mle, load_traindf
from Levenshtein import ratio
from src.madamira.analyse import load_analysis, parse_analyser_output

In [2]:
home = Path(os.getcwd()).parent

In [142]:
train = pd.read_csv(f'{home}/data/processed/train.tsv',delimiter='\t')


In [1]:
# testdf = train[train['ar'].apply(lambda x: 'خريستو' in x)]
# testdf

In [68]:
# locmap = translit_rules.load_loc_mappings()
# exceptional = translit_rules.load_exceptional_spellings()

# tok = 'abc'
# translit_tok = translit_rules.translit_loc_token('يسشابس',locmap,exceptional)
# translit_tok
# ratio(tok,translit_tok)

0.2

In [181]:
# mledf = train_mle(train,size_ratio=0.001)

# of misaligned sentences skipped: 134


In [14]:
dev = pd.read_csv(f'{Path(os.getcwd()).parent}/data/processed/dev.tsv',delimiter='\t')

In [3]:
mledict = load_mle_model(size=1,as_df=True)#mle_model_tsv=f'{home}/models/mle/size1.0.tsv')

In [4]:
mledict[mledict['source']=='تسهيل']

,source,target,freq
8674,تسهيل,Tasʹhīl,14


In [96]:
def apply_lev_ratio(dfrow,col1,col2):
    str1 = str(dfrow[col1]).lower()
    str2 = str(dfrow[col2]).lower()
    return ratio(str1,str2)

In [313]:
deveval = pd.read_csv(f'{home}/evaluation/dev/size1.0.tsv',delimiter='\t')

In [ ]:
deveval[deveval['misaligned']==False]

In [317]:
deveval.value_counts('match_ignore_case',normalize=True)

match_ignore_case
True     0.865683
False    0.134317
dtype: float64

In [73]:
# dev TODO: tweak mle to preserve capitalization only for non-initial toks
testeval = evaluation[evaluation['match_ignore_case']==True]

In [17]:
dev = pd.read_csv(f'{home}/data/processed/dev.tsv',delimiter='\t').set_index('sentID')

In [228]:
apply_mle(testsent['ar'],mledict)

'thawāb al-Aʻmāl wa ;'

In [5]:
translit_morph_predictions = pd.read_csv(f'{home}/predictions_out/translit_rules/dev/translit_morph.tsv',delimiter='\t')

In [ ]:
translit_morph_predictions = pd.read_csv(f'{home}/predictions_out/translit_rules/dev/translit_morph.tsv',delimiter='\t')
def apply_mle_translit_morph_backoff(sentence,backoff_sentences,sent_index):
    predict_mle = []
    for tok_idx, tok in enumerate(tokenize_skiphyph(sentence).split()):
        if tok in mledict:
            mle_tok = mledict[tok]
        else:
            bckf_sent = backoff_sentences[sent_index]
            bckf_tokens = tokenize_skiphyph(bckf_sent).split()
            mle_tok = bckf_tokens[tok_idx]
        predict_mle.append(mle_tok)
    return recompose(' '.join(predict_mle),mode='rom')

In [ ]:
def apply_mle_translit_morph_backoff(sentence,mledict,backoff_sentences,sent_index):
    predict_mle = []
    for tok_idx, tok in enumerate(tokenize_skiphyph(sentence).split()):
        if tok in mledict:
            mle_tok = mledict[tok]
        else:
            bckf_sent = backoff_sentences[sent_index]
            bckf_tokens = tokenize_skiphyph(bckf_sent).split()
            mle_tok = bckf_tokens[tok_idx]
        predict_mle.append(mle_tok)
    return recompose(' '.join(predict_mle),mode='rom')

In [ ]:
translit_morph_predictions = list(translit_morph_predictions[0].values)
    predictions = []
    for sent_idx, sent in enumerate(ar_lines):
        try:
            sent_prediction = apply_mle_translit_morph_backoff(sent,mledict=mledict,backoff_sentences=translit_morph_predictions,sent_index=sent_idx)
        except:
            print(sent)
            print()
            print(translit_morph_predictions[sent_idx])
            exit()
        predictions.append(sent_prediction)

    predictions = pd.DataFrame(predictions)